In [1]:
import numpy as np
import pandas as pd
from misc import load, load_class_labels
from sklearn.metrics import classification_report

predictions_path = "predictions/"
test_info = load("../dataset/labels/test.csv")
class_labels = load_class_labels('../dataset/classes.txt')

def load_predictions(path, model_name):
    predictions = np.load(path + model_name + ".npy")
    names = pd.read_csv(path + model_name + "_names.csv")['filename'].values
    indices = np.argsort(names)
    return predictions[indices], names[indices]

2024-01-14 18:40:35.604106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 18:40:35.604139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 18:40:35.604934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 18:40:35.610007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 18:40:36.529882: W tensorflow/compiler/tf2

In [2]:
vit_predictions, names = load_predictions(predictions_path, "ViT_extended_preprocessed")
en_predictions, _ = load_predictions(predictions_path, "efficientnet_v2_extended_preprocessed")

test_names = test_info['filename'].values
test_labels = test_info['label'].values

ordered_indices = np.argsort(test_names)
ordered_labels = test_labels[ordered_indices]


final_prob = np.zeros(vit_predictions.shape)
for i, (vit_pred, en_pred) in enumerate(zip(vit_predictions, en_predictions)):
    final_prob[i] = np.sqrt(vit_pred * en_pred)
    final_prob[i] /= final_prob[i].sum()
    
final_predictions = np.argmax(final_prob, axis=1)

In [3]:
from tensorflow import keras


print(classification_report(ordered_labels, final_predictions, target_names=class_labels))

                               precision    recall  f1-score   support

                      macaron       0.91      0.95      0.93        55
                      beignet       0.71      0.90      0.80        61
                      cruller       0.83      0.75      0.79        53
                  cockle_food       0.73      0.69      0.71        51
                       samosa       0.70      0.93      0.80        41
                     tiramisu       0.70      0.73      0.72        52
                      tostada       0.72      0.72      0.72        57
                     moussaka       0.56      0.69      0.62        54
                     dumpling       0.61      0.73      0.67        49
                      sashimi       0.78      0.83      0.80        46
                        knish       0.80      0.58      0.67        48
                    croquette       0.54      0.72      0.62        47
                     couscous       0.38      0.73      0.50        45
     

In [4]:
def top_k(predictions, gt, k = 1):
    ohe = keras.utils.to_categorical(gt, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    m.update_state(ohe, predictions)
    return m.result().numpy()

print(top_k(final_prob, ordered_labels, 3))
print(top_k(final_prob, ordered_labels, 1))

2024-01-14 18:40:37.723890: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 18:40:37.904493: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 18:40:37.904702: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

0.8482575
0.68400866


2024-01-14 18:40:39.537188: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24083952 exceeds 10% of free system memory.
2024-01-14 18:40:39.606046: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12041976 exceeds 10% of free system memory.
2024-01-14 18:40:39.634283: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24083952 exceeds 10% of free system memory.
